In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

from custom_estimator import Estimator
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import os
import glob
from sklearn.externals import joblib
from encoding import FreqeuncyEncoding
import warnings
warnings.filterwarnings("ignore")
from sklearn.externals import joblib

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
train=pd.read_pickle('../train_v2.pkl')
test=pd.read_pickle('../test_v2.pkl')

In [3]:
feat1=pd.read_csv('../feats_user_coup_item_no_lag_v2.csv')
feat2=pd.read_csv('../coupon_item_feats_no_lag_v2.csv')
feat3=pd.read_csv('../brand_cust_feats_no_lag_v2.csv')
feat4=pd.read_csv('../brand_feats_no_lag_v2.csv')
feat6=pd.read_csv('../coupon_mean_encoding_v2.csv')
feat7=pd.read_csv('../camp_cust_feats_v2.csv')
feat8=pd.read_csv('../cust_coup_hist3_v2.csv')
feat9=pd.read_csv('../customer_coupon_var1_v2.csv')
feat10=pd.read_csv('../customer_coupon_var2_v2.csv')
feat11=pd.read_csv('../coup_hist1_v2.csv')

In [4]:
train=train.merge(feat1,on='id')
test=test.merge(feat1,on='id')
train=train.merge(feat2,on='id')
test=test.merge(feat2,on='id')
train=train.merge(feat3,on='id')
test=test.merge(feat3,on='id')
train=train.merge(feat4,on='id')
test=test.merge(feat4,on='id')
train=train.merge(feat6,on='coupon_id')
test=test.merge(feat6,on='coupon_id')
train=train.merge(feat7,on=['campaign_id','customer_id'])
test=test.merge(feat7,on=['campaign_id','customer_id'])
train=train.merge(feat8,on=['campaign_id','customer_id'])
test=test.merge(feat8,on=['campaign_id','customer_id'])
train=train.merge(feat9,on=['campaign_id','customer_id'])
test=test.merge(feat9,on=['campaign_id','customer_id'])
train=train.merge(feat10,on=['campaign_id','customer_id'])
test=test.merge(feat10,on=['campaign_id','customer_id'])
train=train.merge(feat11,on=['campaign_id','coupon_id'])
test=test.merge(feat11,on=['campaign_id','coupon_id'])

In [5]:
train.shape

(78369, 188)

In [6]:
target=train[['id','redemption_status']]
train.drop('redemption_status',axis=1,inplace=True)
df=pd.concat((train,test),axis=0)

In [7]:
df.shape

(128595, 187)

In [8]:
df['num_coupons_per_camp']=df['campaign_id'].map(df.groupby('campaign_id')['coupon_id'].nunique().to_dict())
df['num_cust_per_camp']=df['campaign_id'].map(df.groupby('campaign_id')['customer_id'].nunique().to_dict())
df['num_cust_per_coup']=df['coupon_id'].map(df.groupby('coupon_id')['customer_id'].nunique().to_dict())

In [9]:
cat_cols=['campaign_id','coupon_id','customer_id','campaign_type','campaign_start_day','campaign_start_dow',
         'campaign_end_day','campaign_end_dow','item_id',
          'age_range','marital_status','family_size','no_of_children',
         ]
id_cols=['id']
date_cols=['start_date','end_date']

In [10]:
df.head()

,id,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,camp_duration,campaign_start_day,campaign_start_dow,campaign_end_day,campaign_end_dow,days_since_last_campaign_start,days_since_last_campaign_end,days_between_start_end_of_prev_campaign,trans_shape,unique_item,unique_brand,unique_category,item_id,qty_mean,qty_sum,qty_std,qty_max,sp_mean,sp_sum,sp_std,sp_max,od_mean,od_sum,od_std,od_max,coupon_redeemed_mean,coupon_redeemed_sum,coupon_redeemed_std,coupon_redeemed_max,cd_mean,cd_sum,cd_std,cd_max,num_coupons_per_brand_mean,num_coupons_per_brand_sum,num_coupons_per_brand_std,num_coupons_per_brand_max,num_coupons_per_category_mean,num_coupons_per_category_sum,num_coupons_per_category_std,num_coupons_per_category_max,num_coupons_per_item_mean,num_coupons_per_item_sum,num_coupons_per_item_std,num_coupons_per_item_max,cost_price_mean,cost_price_sum,cost_price_std,cost_price_max,selling_price_ewm_mean,selling_price_ewm_sum,selling_price_ewm_std,selling_price_ewm_max,other_discount_ewm_mean,other_discount_ewm_sum,other_discount_ewm_std,other_discount_ewm_max,coupon_discount_ewm_mean,coupon_discount_ewm_sum,coupon_discount_ewm_std,coupon_discount_ewm_max,price_after_OD_ewm_mean,price_after_OD_ewm_sum,price_after_OD_ewm_std,price_after_OD_ewm_max,cost_price_ewm_mean,cost_price_ewm_sum,cost_price_ewm_std,cost_price_ewm_max,coupon_redeemed_ewm_mean,coupon_redeemed_ewm_sum,coupon_redeemed_ewm_std,coupon_redeemed_ewm_max,age_range,marital_status,rented,family_size,no_of_children,income_bracket,num_bought,num_discount_availed,frac_discount_availed,sum_discount,selling_price_sum,coupon_discount_mean,selling_price_mean,other_discount_sum,other_discount_mean,cd_sp_ratio_mean,cd_sp_ratio_sum,num_bought_coup,num_discount_availed_coup,frac_discount_availed_coup,sum_discount_coup,selling_price_sum_coup,coupon_discount_mean_coup,selling_price_mean_coup,other_discount_sum_coup,other_discount_mean_coup,cd_sp_ratio_mean_coup,cd_sp_ratio_sum_coup,num_bought_brand_cust,num_discount_availed_brand_cust,frac_discount_availed_brand_cust,sum_discount_brand_cust,selling_price_sum_brand_cust,coupon_discount_mean_brand_cust,selling_price_mean_brand_cust,other_discount_sum_brand_cust,other_discount_mean_brand_cust,cd_sp_ratio_mean_brand_cust,cd_sp_ratio_sum_brand_cust,num_bought_brand,num_discount_availed_brand,frac_discount_availed_brand,sum_discount_brand,selling_price_sum_brand,coupon_discount_mean_brand,selling_price_mean_brand,other_discount_sum_brand,other_discount_mean_brand,cd_sp_ratio_mean_brand,cd_sp_ratio_sum_brand,tot_redemption,mean_redemption,num_bought_camp_cust,num_discount_availed_camp_cust,frac_discount_availed_camp_cust,sum_discount_camp_cust,selling_price_sum_camp_cust,coupon_discount_mean_camp_cust,selling_price_mean_camp_cust,other_discount_sum_camp_cust,other_discount_mean_camp_cust,cd_sp_ratio_mean_camp_cust,cd_sp_ratio_sum_camp_cust,num_bought_cust_var3,num_discount_availed_cust_var3,frac_discount_availed_cust_var3,sum_discount_cust_var3,selling_price_sum_cust_var3,coupon_discount_mean_cust_var3,selling_price_mean_cust_var3,other_discount_sum_cust_var3,other_discount_mean_cust_var3,cd_sp_ratio_mean_cust_var3,cd_sp_ratio_sum_cust_var3,num_bought_cust_var1,num_discount_availed_cust_var1,frac_discount_availed_cust_var1,sum_discount_cust_var1,selling_price_sum_cust_var1,coupon_discount_mean_cust_var1,selling_price_mean_cust_var1,other_discount_sum_cust_var1,other_discount_mean_cust_var1,cd_sp_ratio_mean_cust_var1,cd_sp_ratio_sum_cust_var1,num_bought_cust_var2,num_discount_availed_cust_var2,frac_discount_availed_cust_var2,sum_discount_cust_var2,selling_price_sum_cust_var2,coupon_discount_mean_cust_var2,selling_price_mean_cust_var2,other_discount_sum_cust_var2,other_discount_mean_cust_var2,cd_sp_ratio_mean_cust_var2,cd_sp_ratio_sum_cust_var2,num_bought_coup_var1,num_discount_availed_coup_var1,frac_discount_availed_coup_var1,sum_discount_coup_var1,selling_price_sum_coup_var1,coupon_discount_mean_coup_var1,selling_price_mean_co

In [11]:
from sklearn.model_selection import GroupKFold
folds=GroupKFold(n_splits=5)
temp=df[df.id.isin(train.id)]
folds1=[(x,y) for x,y in  folds.split(temp,temp.merge(target,on='id').redemption_status,temp.coupon_id)]
folds2=[(x,y) for x,y in  folds.split(temp,temp.merge(target,on='id').redemption_status,temp.campaign_id)]

In [12]:
fe=FreqeuncyEncoding(normalize=1,return_df=True,categorical_columns=cat_cols)
df=fe.fit_transform(df)

In [13]:
use_cols=df.columns[~df.columns.isin(date_cols+id_cols)].tolist()

In [14]:
train_df=df[df.id.isin(train.id)]
test_df=df[df.id.isin(test.id)]
train_df.shape,test_df.shape

((78369, 190), (50226, 190))

In [15]:
train_df=train_df.merge(target,on='id')

In [16]:
params2={'boosting_type': 'gbdt',
 'colsample_bytree': 0.30000000000000004,
 'learning_rate': 0.1,
 'metric': 'None',
 'min_child_weight': 10.0,
 'n_estimators': 5000,
 'n_jobs': -1,
 'num_leaves': 112,
 'objective': 'binary',
 'subsample': 0.9,
 'subsample_freq': 5}
est_lgb=Estimator(model=LGBMClassifier(**params2),n_jobs=-1,early_stopping_rounds=300,
              validation_scheme=folds1
                 )

In [17]:
oof_lgb=est_lgb.fit_transform(train_df[use_cols].values,train_df['redemption_status'].values)
est_lgb.avg_cv_score

Training until validation scores don't improve for 300 rounds
[100]	valid_0's auc: 0.95024	valid_1's auc: 0.998234
[200]	valid_0's auc: 0.950959	valid_1's auc: 0.999909
[300]	valid_0's auc: 0.951801	valid_1's auc: 0.999998
Early stopping, best iteration is:
[62]	valid_0's auc: 0.953382	valid_1's auc: 0.994467
Training until validation scores don't improve for 300 rounds
[100]	valid_0's auc: 0.959214	valid_1's auc: 0.998509
[200]	valid_0's auc: 0.95662	valid_1's auc: 0.999937
[300]	valid_0's auc: 0.954304	valid_1's auc: 0.999996
Early stopping, best iteration is:
[79]	valid_0's auc: 0.961587	valid_1's auc: 0.997137
Training until validation scores don't improve for 300 rounds
[100]	valid_0's auc: 0.961852	valid_1's auc: 0.99802
[200]	valid_0's auc: 0.957694	valid_1's auc: 0.99983
[300]	valid_0's auc: 0.957485	valid_1's auc: 0.999959
Early stopping, best iteration is:
[54]	valid_0's auc: 0.965372	valid_1's auc: 0.993159
Training until validation scores don't improve for 300 rounds
[100]	

0.9595300439653738

In [18]:
train_df[['id','redemption_status']].to_csv('meta_target.csv',index=False)

In [19]:
est_lgb.feature_importance_df(train_df[use_cols].columns)

,column,feature_importance,rank
125,cd_sp_ratio_mean_brand,0.013286,1
87,selling_price_sum,0.013173,2
128,mean_redemption,0.012422,3
136,other_discount_sum_camp_cust,0.011600,4
109,selling_price_sum_brand_cust,0.011594,5
118,frac_discount_availed_brand,0.010941,6
122,selling_price_mean_brand,0.010884,7
111,selling_price_mean_brand_cust,0.010586,8
14,unique_brand,0.010578,9
170,other_discount_mean_cust_var2,0.010561,10


In [20]:
sub=pd.read_csv('../sample_submission_Byiv0dS.csv')

In [21]:
preds=est_lgb.transform(test_df[use_cols].values)

In [22]:
pd.DataFrame({'id':train_df['id'],'redemption_status': oof_lgb}).to_csv('OOF_baseline_lgb_grp_k_coup_more_feats_v2.csv',index=False)

In [23]:
pd.DataFrame({'id':test_df['id'],'redemption_status': preds}).to_csv('baseline_lgb_grp_k_coup_more_feats_v2.csv',index=False)